In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:

class NeuralNet_ont_hot(nn.Module):
    def __init__(self):
        super(NeuralNet_ont_hot, self).__init__()
        self.fc1 = nn.Linear(784, 15)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(15, 10)
        self.type = 'MLP'

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        out = self.fc2(x)
        return out


class NeuralNet_binary_encoding(nn.Module):
    def __init__(self):
        super(NeuralNet_binary_encoding, self).__init__()
        self.fc1 = nn.Linear(784, 15)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(15, 4)
        self.type = 'MLP'

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        out = self.fc2(x)
        return out
    
class NeuralNet_binary_encoding_one_more_layer(nn.Module):
    def __init__(self):
        super(NeuralNet_binary_encoding_one_more_layer, self).__init__()
        self.fc1 = nn.Linear(784, 15)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(15, 4)
        self.fc3 = nn.Linear(4, 4)
        self.type = 'MLP'

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        out = self.fc3(x)
        return out
    
    
def create_dataloader_one_hot():
    
    # MNIST dataset
    train_dataset = torchvision.datasets.MNIST(root='/',
                                            train=True,
                                            download=True,
                                            transform=transforms.ToTensor(),
                                            target_transform=lambda y: torch.zeros(10,dtype=torch.float).scatter_(0, torch.tensor(y),value=1))

    test_dataset = torchvision.datasets.MNIST(root='/',
                                            train=False,
                                            download=True,
                                            transform=transforms.ToTensor(),
                                            target_transform=lambda y: torch.zeros(10,dtype=torch.float).scatter_(0, torch.tensor(y),value=1))

    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=64,
                                            shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=64,
                                            shuffle=False)

    return train_loader, test_loader



def create_dataloader_binary_encoding():
    
    # MNIST dataset
    train_dataset = torchvision.datasets.MNIST(root='/',
                                                train=True,
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform = lambda y: np.array(list(np.binary_repr(y).zfill(4))).astype(np.float32))

    test_dataset = torchvision.datasets.MNIST(root='/',
                                                train=False,
                                                download=True,
                                                transform=transforms.ToTensor(),
                                                target_transform = lambda y: np.array(list(np.binary_repr(y).zfill(4))).astype(np.float32))
    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=64,
                                            shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size=64,
                                            shuffle=False)

    return train_loader, test_loader





def train(train_loader, model, criterion, optimizer, num_epochs):
    # Train the model
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for step, (images, labels) in enumerate(train_loader):
            if model.type == 'MLP':
                images = images.reshape(-1, 28 * 28)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs, step + 1, total_step, loss.item()))

def test(test_loader, model):
    # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            if model.type == 'MLP':
                images = images.reshape(-1, 28 * 28)
            outputs = model(images)
            
            predicted = np.abs(np.round(outputs.numpy()))
            i = 0
            for answer in labels.numpy():
                if (predicted[i] == answer).all():
                    correct += 1
                i += 1
            total += labels.size(0)
            #correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


# one_hot

In [3]:
train_loader, test_loader = create_dataloader_one_hot()

In [4]:
model = NeuralNet_ont_hot()

In [5]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
train(train_loader, model, criterion, optimizer, num_epochs=5)

Epoch [1/5], Step [100/938], Loss: 0.0526
Epoch [1/5], Step [200/938], Loss: 0.0462
Epoch [1/5], Step [300/938], Loss: 0.0391
Epoch [1/5], Step [400/938], Loss: 0.0357
Epoch [1/5], Step [500/938], Loss: 0.0312
Epoch [1/5], Step [600/938], Loss: 0.0257
Epoch [1/5], Step [700/938], Loss: 0.0288
Epoch [1/5], Step [800/938], Loss: 0.0309
Epoch [1/5], Step [900/938], Loss: 0.0253
Epoch [2/5], Step [100/938], Loss: 0.0258
Epoch [2/5], Step [200/938], Loss: 0.0239
Epoch [2/5], Step [300/938], Loss: 0.0173
Epoch [2/5], Step [400/938], Loss: 0.0280
Epoch [2/5], Step [500/938], Loss: 0.0229
Epoch [2/5], Step [600/938], Loss: 0.0205
Epoch [2/5], Step [700/938], Loss: 0.0175
Epoch [2/5], Step [800/938], Loss: 0.0217
Epoch [2/5], Step [900/938], Loss: 0.0265
Epoch [3/5], Step [100/938], Loss: 0.0174
Epoch [3/5], Step [200/938], Loss: 0.0213
Epoch [3/5], Step [300/938], Loss: 0.0200
Epoch [3/5], Step [400/938], Loss: 0.0188
Epoch [3/5], Step [500/938], Loss: 0.0198
Epoch [3/5], Step [600/938], Loss:

In [7]:
test(test_loader, model)

Accuracy of the network on the 10000 test images: 86.66 %


# binary_encoding

In [8]:
train_loader, test_loader = create_dataloader_binary_encoding()
model = NeuralNet_binary_encoding()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
train(train_loader, model, criterion, optimizer, num_epochs=5)

Epoch [1/5], Step [100/938], Loss: 0.0945
Epoch [1/5], Step [200/938], Loss: 0.0814
Epoch [1/5], Step [300/938], Loss: 0.0683
Epoch [1/5], Step [400/938], Loss: 0.0758
Epoch [1/5], Step [500/938], Loss: 0.0676
Epoch [1/5], Step [600/938], Loss: 0.0617
Epoch [1/5], Step [700/938], Loss: 0.0522
Epoch [1/5], Step [800/938], Loss: 0.0473
Epoch [1/5], Step [900/938], Loss: 0.0545
Epoch [2/5], Step [100/938], Loss: 0.0665
Epoch [2/5], Step [200/938], Loss: 0.0547
Epoch [2/5], Step [300/938], Loss: 0.0420
Epoch [2/5], Step [400/938], Loss: 0.0476
Epoch [2/5], Step [500/938], Loss: 0.0555
Epoch [2/5], Step [600/938], Loss: 0.0489
Epoch [2/5], Step [700/938], Loss: 0.0362
Epoch [2/5], Step [800/938], Loss: 0.0638
Epoch [2/5], Step [900/938], Loss: 0.0610
Epoch [3/5], Step [100/938], Loss: 0.0565
Epoch [3/5], Step [200/938], Loss: 0.0400
Epoch [3/5], Step [300/938], Loss: 0.0532
Epoch [3/5], Step [400/938], Loss: 0.0452
Epoch [3/5], Step [500/938], Loss: 0.0501
Epoch [3/5], Step [600/938], Loss:

In [10]:
test(test_loader, model)

Accuracy of the network on the 10000 test images: 88.68 %


# binary_encoding with one more layer

In [11]:
train_loader, test_loader = create_dataloader_binary_encoding()
model = NeuralNet_binary_encoding_one_more_layer()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
train(train_loader, model, criterion, optimizer, num_epochs=5)

Epoch [1/5], Step [100/938], Loss: 0.2254
Epoch [1/5], Step [200/938], Loss: 0.1393
Epoch [1/5], Step [300/938], Loss: 0.1193
Epoch [1/5], Step [400/938], Loss: 0.1005
Epoch [1/5], Step [500/938], Loss: 0.0902
Epoch [1/5], Step [600/938], Loss: 0.0924
Epoch [1/5], Step [700/938], Loss: 0.0932
Epoch [1/5], Step [800/938], Loss: 0.0812
Epoch [1/5], Step [900/938], Loss: 0.0833
Epoch [2/5], Step [100/938], Loss: 0.0585
Epoch [2/5], Step [200/938], Loss: 0.0930
Epoch [2/5], Step [300/938], Loss: 0.0721
Epoch [2/5], Step [400/938], Loss: 0.0691
Epoch [2/5], Step [500/938], Loss: 0.0789
Epoch [2/5], Step [600/938], Loss: 0.0779
Epoch [2/5], Step [700/938], Loss: 0.0872
Epoch [2/5], Step [800/938], Loss: 0.0663
Epoch [2/5], Step [900/938], Loss: 0.0678
Epoch [3/5], Step [100/938], Loss: 0.0669
Epoch [3/5], Step [200/938], Loss: 0.0752
Epoch [3/5], Step [300/938], Loss: 0.0831
Epoch [3/5], Step [400/938], Loss: 0.0642
Epoch [3/5], Step [500/938], Loss: 0.0581
Epoch [3/5], Step [600/938], Loss:

In [13]:
test(test_loader, model)

Accuracy of the network on the 10000 test images: 86.31 %
